In [15]:
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
from time import time_ns
from amftrack.util.dbx import upload_folders, load_dbx, download, get_dropbox_folders
from datetime import datetime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
plates  = ['19_20220920']

In [44]:
folders_drop = all_folders_drop.loc[
    all_folders_drop["folder"] == "20221116_1111_Plate21"
]

In [47]:
run_parallel_transfer(
    "from_drop.py",
    [directory_targ],
    folders_drop,
    50,
    "24:00:00",
    "staging",
    cpus=1,
    node="staging",
    name_job="kaikai",
)

In [10]:
all_folders_drop = get_dropbox_folders("/DATA/PRINCE", True)

2022-11-04 10:13:53,458-[INFO]- dropbox:390 -> Refreshing access token.
2022-11-04 10:13:53,719-[INFO]- dropbox:474 -> Request to files/list_folder


KeyboardInterrupt: 

In [43]:
dir_drop = "DATA/PRINCE"
all_folders_drop = get_dropbox_folders("/DATA/PRINCE", True)
folders_drop = all_folders_drop.loc[all_folders_drop["unique_id"].isin(plates)]
update_plate_info(directory_targ, local=True, strong_constraint=False)
all_folders = get_current_folders(directory_targ, local=True)
# if len(all_folders) > 0:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders_drop2 = folders_drop.loc[~folders_drop["folder"].isin(folders["folder"])]
folders_drop3 = folders_drop2.loc[~folders_drop2["folder"].str.contains("Analysis")]

2022-11-16 18:06:19,438-[INFO]- dropbox:390 -> Refreshing access token.
2022-11-16 18:06:19,697-[INFO]- dropbox:474 -> Request to files/list_folder
2022-11-16 18:06:54,157-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2022-11-16 18:06:59,192-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2022-11-16 18:07:04,397-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2022-11-16 18:07:10,025-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2022-11-16 18:07:15,111-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2022-11-16 18:07:20,073-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2022-11-16 18:07:25,146-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2022-11-16 18:07:30,149-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2022-11-16 18:07:34,931-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2022-11-16 18:07:39,911-[INFO]- dropbox:474 -> Request to files/list_folder/continue
20

analysed:   0%|          | 0/578 [00:00<?, ?it/s]

In [39]:
folders_drop3

,folder,size,change_date,tot_path_drop,Plate,CrossDate,unique_id


In [8]:
on_dropbox = all_folders_drop.loc[
    all_folders_drop["CrossDate"].between("20220820", "20220901")
]
plates = on_dropbox["unique_id"].unique()

In [9]:
plates

array(['109_20220829', '140_20220826', '106_20220826', '46_20220829',
       '111_20220823', '107_20220821', '108_20220823', '66_20220823',
       '996_20220823'], dtype=object)

In [10]:
len(on_dropbox)

2847

In [2]:
# plates= ['94_20201123','102_20201226','76_20210108','152_20201224','792_20210915']
# plates= ['52_20220707','56_20220701','839_20220325','938_20220325','1076_20220511']
plates = [
    "52_20220707",
    "56_20220701",
    "81_20220704",
    "89_20220705",
    "91_20220708",
    "839_20220325",
    "1076_20220511",
]  # full run done!
plates = [
    "84_20220621",
    "877_20220419",
    "938_20220325",
    "941_20220404",
]  # for spore data

# plates = ['52_20220707','84_20220621','1048_20220506','1045_20220504']
plates = [
    # "20_20220419",
    # "23_20220509",
    # "24_20220426",
    # "30_20220509",
    # "32_20220428",
    "33_20220502",
]  # analysis AGG
# plates = ["20_20220419", "23_20220509", "24_20220426"]  # analysis AGG

# plates = ['111111_20600101']
# plates = ["56_20220701","839_20220325",'938_20220325','1076_20220511'] #analysis A5
# plates = ["81_20220704","84_20220621","89_20220705","91_20220708"] #analysis A5sp3
# plates = ["127_20220705","129_20220711","130_20220707","1014_20220525","1023_20220502","1048_20220506", "1045_20220504","1035_20220518","1037_20220523","1042_20220523","1053_20220413"] #analysis C2
# plates = ["1035_20220518","1037_20220523","1042_20220523","1053_20220413"] #analysis C2
# plates = ['94_20201123','152_20201224','792_20210915']
# plates = ['799_20211001','800_20210928','91_20201217']
# plates = ["1044_20220509"]
# plates = ["1048_20220506", "1045_20220504", "52_20220707", "84_20220621"]
# plates = [
#     '102_20201226',
#      '94_20201123',
#     "1076_20220511",
#     "1042_20220523",
#     "1048_20220506",
#     "938_20220325",
#          ]
# plates = ['907_20220321','943_20220322',"403_20221005"] #for philippe
# plates = ['799_20211001','800_20210928',"91_20201217"]
plates = [
    "106_20220826",
    "108_20220823",
    "111_20220823",
    "66_20220823",
    "105_20220817",
    "100_20220812",
    "75_20220808",
    "99_20220815",
]

plates = ["452_20221017"]  # for spore data
# plates = ['1048_20220506','84_20220621'] #for spore data
# plates = ['510_20220623']

In [24]:
plates = [
    "078_20220920",
    "079_20220920",
    "080_20220920",
    "081_20220920",
    "082_20220920",
    "083_20220920",
    "084_20220920",
    "001_20220920",
    "024_20220920",
    "073_20220920",
    "002_20220920",
    "003_20220920",
    "004_20220920",
    "005_20220920",
    "006_20220920",
    "007_20220920",
    "008_20220920",
    "009_20220920",
    "010_20220920",
    "011_20220920",
    "012_20220920",
    "019_20220920",
    "020_20220920",
    "021_20220920",
    "022_20220920",
    "023_20220920",
    "037_20220920",
    "038_20220920",
    "039_20220920",
    "040_20220920",
    "042_20220920",
    "043_20220920",
    "044_20220920",
    "045_20220920",
    "046_20220920",
    "016_20220921",
    "017_20220921",
    "034_20220921",
    "051_20220920",
    "094_20220921",
    "064_20220921",
    "026_20220921",
    "027_20220921",
    "028_20220921",
    "029_20220921",
    "031_20220921",
    "033_20220921",
    "035_20220921",
    "036_20220921",
    "047_20220920",
    "041_20220920",
    "015_20220921",
    "068_20220921",
    "107_20220921",
    "095_20220921",
    "062_20220921",
    "096_20220921",
    "104_20220921",
    "050_20220920",
    "030_20220921",
    "025_20220921",
    "100_20220921",
    "067_20220921",
    "048_20220920",
    "055_20220921",
    "056_20220921",
    "057_20220921",
    "058_20220921",
    "059_20220921",
    "060_20220921",
    "061_20220921",
    "063_20220921",
    "014_20220921",
    "018_20220921",
    "099_20220921",
    "069_20220921",
    "054_20220920",
    "075_20220920",
    "092_20220921",
    "093_20220921",
    "097_20220921",
    "098_20220921",
    "105_20220921",
    "106_20220921",
    "108_20220921",
    "085_20220921",
    "086_20220921",
    "087_20220921",
    "088_20220921",
    "089_20220921",
    "090_20220921",
    "091_20220921",
    "049_20220920",
    "052_20220920",
    "053_20220920",
    "074_20220920",
    "076_20220920",
    "077_20220920",
    "013_20220921",
    "070_20220921",
    "071_20220921",
    "072_20220921",
    "102_20220921",
    "032_20220921",
    "101_20220921",
    "065_20220921",
    "066_20220921",
    "103_20220921",
]
plates = [
    "_".join((str(int(plate.split("_")[0])), str(int(plate.split("_")[1]))))
    for plate in plates
]

In [28]:
plates

['78_20220920',
 '79_20220920',
 '80_20220920',
 '81_20220920',
 '82_20220920',
 '83_20220920',
 '84_20220920',
 '1_20220920',
 '24_20220920',
 '73_20220920',
 '2_20220920',
 '3_20220920',
 '4_20220920',
 '5_20220920',
 '6_20220920',
 '7_20220920',
 '8_20220920',
 '9_20220920',
 '10_20220920',
 '11_20220920',
 '12_20220920',
 '19_20220920',
 '20_20220920',
 '21_20220920',
 '22_20220920',
 '23_20220920',
 '37_20220920',
 '38_20220920',
 '39_20220920',
 '40_20220920',
 '42_20220920',
 '43_20220920',
 '44_20220920',
 '45_20220920',
 '46_20220920',
 '16_20220921',
 '17_20220921',
 '34_20220921',
 '51_20220920',
 '94_20220921',
 '64_20220921',
 '26_20220921',
 '27_20220921',
 '28_20220921',
 '29_20220921',
 '31_20220921',
 '33_20220921',
 '35_20220921',
 '36_20220921',
 '47_20220920',
 '41_20220920',
 '15_20220921',
 '68_20220921',
 '107_20220921',
 '95_20220921',
 '62_20220921',
 '96_20220921',
 '104_20220921',
 '50_20220920',
 '30_20220921',
 '25_20220921',
 '100_20220921',
 '67_20220921'

In [5]:
import imageio.v2 as imageio

In [52]:
folders_drop = all_folders_drop.loc[
    all_folders_drop["unique_id"].isin(select_plates)
].loc[all_folders_drop["folder"].str.split("_", expand=True)[0] == "20220809"]

In [3]:
directory_targ = directory_project
directory_project

'/projects/0/einf914/data/'

In [5]:
directory_targ = os.path.join(directory_scratch, "stitch_temp2") + "/"

In [4]:
name_job = f"one_shot_{time_ns()}.sh"
run_launcher(
    "dropbox_downloader.py",
    [directory_targ, name_job, -1, "stitcher.py"],
    plates,
    "12:00:00",
    name_job=name_job,
)
# run_launcher('dropbox_uploader.py',[directory_targ,name_job],plates,'40:00',dependency=True,name_job = name_job)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


Submitted batch job 1851737


In [4]:
name_job = f"one_shot_{time_ns()}.sh"
run_launcher(
    "dropbox_downloader.py",
    [directory_targ, name_job, 12, "post_processer_1.py"],
    plates,
    "24:00:00",
    name_job=name_job,
)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


Submitted batch job 1814579


In [12]:
max((3000, 4096))

4096

In [25]:
plates

['1048_20220506']

In [61]:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Img/TileConfiguration.txt.registered"] == True]
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime
10,/projects/0/einf914/data/20220601_1143_Plate12,30,12,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"01.06.2022, 11:43:",20220601_1143_Plate12,30_20220509,2022-06-01 11:43:00
23,/projects/0/einf914/data/20220618_0150_Plate11,23,11,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"18.06.2022, 01:50:",20220618_0150_Plate11,23_20220509,2022-06-18 01:50:00
24,/projects/0/einf914/data/20220528_1900_Plate12,30,12,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"28.05.2022, 19:00:",20220528_1900_Plate12,30_20220509,2022-05-28 19:00:00
25,/projects/0/einf914/data/20220619_0156_Plate11,23,11,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"19.06.2022, 01:56:",20220619_0156_Plate11,23_20220509,2022-06-19 01:56:00
29,/projects/0/einf914/data/20220613_0147_Plate12,30,12,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"13.06.2022, 01:47:",20220613_0147_Plate12,30_20220509,2022-06-13 01:47:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4882,/projects/0/einf914/data/20220529_0302_Plate12,30,12,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"29.05.2022, 03:02:",20220529_0302_Plate12,30_20220509,2022-05-29 03:02:00
4886,/projects/0/einf914/data/20220604_0131_Plate11,23,11,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"04.06.2022, 01:31:",20220604_0131_Plate11,23_20220509,2022-06-04 01:31:00
4889,/projects/0/einf914/data/20220619_1955_Plate11,23,11,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"19.06.2022, 19:55:",20220619_1955_Plate11,23_20220509,2022-06-19 19:55:00
4890,/projects/0/einf914/data/20220616_1904_Plate10,22,10,'Carrot','Agg','001P100N100C','Y','-',20220522,25,...,False,False,False,False,False,False,"16.06.2022, 19:04:",20220616_1904_Plate10,22_20220522,2022-06-16 19:04:00


In [4]:
run_launcher("skeletonizer.py", [directory_targ], plates, "20:00", dependency=True)

Submitted batch job 1129540


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 245760 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


In [9]:
plates

['106_20220826',
 '108_20220823',
 '111_20220823',
 '66_20220823',
 '105_20220817',
 '100_20220812',
 '75_20220808',
 '99_20220815']

In [5]:
name_job = "kaikai"

# name_job = f"one_shot_{time_ns()}.sh"

run_launcher(
    "stitcher.py",
    [directory_targ, name_job, -1],
    plates,
    "12:00:00",
    dependency=True,
    name_job=name_job,
)

Submitted batch job 1851741


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


In [5]:
name_job = f"one_shot_{time_ns()}.sh"
run_launcher(
    "post_processer_1.py",
    [directory_targ, name_job, 10],
    plates,
    "20:00",
    dependency=True,
    name_job=name_job,
)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


Submitted batch job 1719140


In [4]:
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)
# folders =all_folders.loc[all_folders["unique_id"].isin(plates)]
# folders = folders.loc[folders["/Analysis/skeleton_pruned_compressed.mat"]==True]

analysed:   0%|          | 0/578 [00:00<?, ?it/s]

In [17]:
plates  = ['19_20220920']

In [25]:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
len(folders)

555

In [26]:
num_parallel = 50
time = "40:00"
if len(folders) > 0:
    run_parallel_stitch(
        directory_targ,
        folders,
        num_parallel,
        time,
        cpus=128,
        node="fat",
        name_job="stitch_malin",
        is_mini_PRINCE = True
    )

Submitted batch job 1851999
Submitted batch job 1852000
Submitted batch job 1852001
Submitted batch job 1852002
Submitted batch job 1852003
Submitted batch job 1852004
Submitted batch job 1852005
Submitted batch job 1852006
Submitted batch job 1852007
Submitted batch job 1852008
Submitted batch job 1852009
Submitted batch job 1852010


In [14]:
1e9/(3600*24*365)

31.709791983764585

In [5]:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]

folders = folders.loc[folders["/Analysis/skeleton_compressed.mat"] == False]
len(folders)

0

In [12]:
folders = folders.iloc[0:1]

In [6]:
folders

,total_path,Plate,strain,medium,CrossDate,/Img/TileConfiguration.txt.registered,/Analysis/skeleton_compressed.mat,/Analysis/skeleton_masked_compressed.mat,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,...,/Analysis/nx_graph_pruned_labeled.p,date,folder,PrincePos,root,split,Pbait,Temperature,unique_id,datetime


In [6]:
update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]

/projects/0/einf914/data/Analysis_33_20220502_188_Version1/folder_info.json
/projects/0/einf914/data/Analysis_105_20220817_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_105_20220817_5_Version1/folder_info.json


In [7]:
analysis_folders

,folder_analysis,version,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,...,datetime,date_begin,date_end,number_timepoints,path_exp,path_global_hypha_info,path_time_hypha_info,path_time_plate_info,path_global_plate_info,date_run_analysis
0,Analysis_33_20220502_188_Version1,Version1,/projects/0/einf914/data/20220502_2111_Plate07,33,7,'Carrot','Agg','001P100N100C','Y','-',...,2022-05-02 21:11:00,"02.05.2022, 21:11:","20.05.2022, 09:41:",188,Analysis_33_20220502_188_Version1/experiment.pick,Analysis_33_20220502_188_Version1/global_hypha...,Analysis_33_20220502_188_Version1/time_hypha_info,Analysis_33_20220502_188_Version1/time_plate_i...,Analysis_33_20220502_188_Version1/global_plate...,"01.01.1970, 01:00:"


In [8]:
# name_job = f"one_shot_{time_ns()}.sh"

from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.global_plate import *
from amftrack.pipeline.functions.post_processing.time_plate import *
from amftrack.pipeline.functions.post_processing.global_hypha import *
from amftrack.pipeline.functions.post_processing.area_hulls import *
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher
from amftrack.pipeline.functions.post_processing.exp_plot import *

time = "24:00:00"
directory = directory_targ
max_ind = 20
incr = 100
fs = [
    get_density_in_ring,
    get_density_anastomose_in_ring,
    # get_density_branch_rate_in_ring,
    # get_density_stop_rate_in_ring,
    get_density_active_tips_in_ring,
]
# fs = [get_mean_speed_in_ring]

list_f = []
list_args = []

for f in fs:
    list_f += [f] * max_ind

    list_args += [{"incr": incr, "i": i, "rh_only": True} for i in range(max_ind)]
overwrite = False
num_parallel = 30
run_parallel_post(
    "time_plate_post_process.py",
    list_f,
    list_args,
    [directory, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "time_plate_post_process",
    cpus=32,
    name_job=name_job,
    node="fat",
    dependency=True,
)
run_launcher(
    "analysis_uploader.py",
    [directory_targ, name_job],
    plates,
    "12:00:00",
    dependency=True,
    name_job=name_job,
)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1768604


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


Submitted batch job 1768605


In [21]:
from amftrack.pipeline.launching.run_super import run_parallel_all_time, run_launcher

num_parallel = 30
time = "1:00:00"
args = []
run_parallel_all_time(
    "make_video_skelet.py",
    args,
    folders,
    num_parallel,
    time,
    "make_video",
    cpus=32,
    node="fat",
    dependency=False,
    name_job=name_job,
)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1832161


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1832162


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1832164


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1832165


In [9]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

num_parallel = 100
time = "12:00:00"
hyph_width = 30
perc_low = 93
perc_high = 99.5
args = [hyph_width, perc_low, perc_high, directory_targ]
run_parallel(
    "extract_skel_2.py",
    args,
    folders,
    num_parallel,
    time,
    "skeletonization",
    cpus=128,
    node="fat",
    name_job="test",
)

Submitted batch job 1834117
Submitted batch job 1834118
Submitted batch job 1834119
Submitted batch job 1834120
Submitted batch job 1834121
Submitted batch job 1834122


In [9]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

num_parallel = 100
time = "20:00"
args = [directory_targ]
run_parallel(
    "detect_blob.py",
    args,
    folders,
    num_parallel,
    time,
    "detect_blob",
    cpus=128,
    node="fat",
    name_job="spore",
)

Submitted batch job 1828162


In [24]:
!pip install statsmodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 81.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.8/233.8 kB 27.9 MB/s eta 0:00:00


In [25]:
from amftrack.pipeline.launching.run_super import run_parallel_all_time, run_launcher

folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Img/TileConfiguration.txt.registered"] == True]
num_parallel = 50
time = "10:00"
args = []
run_parallel_all_time(
    "make_video_stitched.py",
    args,
    folders,
    num_parallel,
    time,
    "make_video",
    cpus=32,
    node="fat",
    dependency=False,
    name_job="video",
)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1828129


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1828130


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1828131


In [3]:
plates = [
    "105_20220817",
    "77_20220808",
    "71_20220817",
    "100_20220812",
    "117_20220815",
    "99_20220815",
    "75_20220808",
    "226_20220812",
]

In [46]:
folders = all_folders.loc[all_folders["Plate"].isin(["32"])]

In [17]:
from amftrack.pipeline.launching.run_super import *

num_parallel = 32
time = "8:00:00"
limit = 200
version = 1
labeled = True
args = [directory_targ, limit, version, labeled]
run_parallel_all_time(
    "hyphae_extraction.py",
    args,
    folders,
    num_parallel,
    time,
    "hyphae",
    cpus=32,
    node="fat",
)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1738329


In [12]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

for plate in plates:
    select = folders.loc[folders["Plate"] == plate]
    num_parallel = 128
    time = "12:00:00"
    thresh = 10000  # For R. irregularis, thresh 10000 is good. For Aggregatum, higher may be necessary
    args = [thresh, directory_targ]
    run_parallel(
        "final_alignment.py",
        args,
        select,
        num_parallel,
        time,
        "realign",
        cpus=128,
        node="fat",
        name_job=name_job,
    )

Submitted batch job 1138954


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [20]:
plates

['6_20220509']

In [19]:
select

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime


In [13]:
i = 0
run_info = folders
folder_list = list(run_info["folder"])
folder_list.sort()

dates_datetime_chosen = folder_list[i : i + 2]
print("========")
print(f"Matching plate {dates_datetime_chosen[0]} at dates {dates_datetime_chosen}")
print("========")
dates = dates_datetime_chosen

Matching plate 20220510_1851_Plate17 at dates ['20220510_1851_Plate17', '20220511_1750_Plate17']


In [10]:
from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.global_plate import *
from amftrack.pipeline.functions.post_processing.time_plate import *
from amftrack.pipeline.functions.post_processing.global_hypha import *
from amftrack.pipeline.functions.post_processing.area_hulls import *
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher
from amftrack.pipeline.functions.post_processing.exp_plot import *

update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
directory = directory_targ

In [11]:
time = "3:40:00"
directory = directory_targ
# list_f = [get_width_f,get_tot_length_C_f,get_tot_growth_C_f]
# list_f = [get_timestep_anastomosis]
list_f = [
    gets_out_of_ROI,
    get_width_f,
    get_tot_length_C_f,
    get_tot_growth_C_f,
    get_tot_length_pp_f,
    get_tot_growth_pp_f,
    get_timestep_stop_growth,
    get_time_stop_growth,
    get_time_init_growth,
    get_mean_speed_growth,
    get_stop_track,
    get_timestep_anastomosis,
    get_timestep_biological_stop_growth,
]
# list_f = [get_num_branch]
# list_f = [gets_out_of_ROI]
list_args = [{}] * len(list_f)
overwrite = False
num_parallel = 32
run_parallel_post(
    "global_hypha_post_process.py",
    list_f,
    list_args,
    [directory, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "global_hypha_post_process",
    cpus=32,
    name_job="a",
    node="fat",
)

Submitted batch job 1539552


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [13]:
import sys
from amftrack.util.sys import (
    update_analysis_info,
    get_analysis_info,
)
from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher
import pandas as pd
import os

list_f = [
    get_pos_x,
    get_pos_y,
    get_time_since_begin_exp,
    get_distance_final_pos,
    get_timedelta,
    get_time_since_start,
    get_speed,
    get_timestep,
    get_timestep_init,
    get_time_init,
    get_degree,
    get_width_tip_edge,
    get_width_root_edge,
    get_width_average,
    has_reached_final_pos,
    in_ROI,
]
# list_f = [local_density,local_density,local_density]
# list_f = [get_time_since_begin_exp]
# list_f = [get_width_tip_edge, get_width_root_edge]
list_args = [{}] * len(list_f)
# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = False
load_graphs = True
num_parallel = 32
time = "12:00:00"
for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post(
            "time_hypha_post_process.py",
            list_f,
            list_args,
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "time_hypha_post_process",
            cpus=num_cpus,
            name_job=name_job,
            node="fat",
        )

Submitted batch job 1539625
Submitted batch job 1539626
Submitted batch job 1539627


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memor

Submitted batch job 1539628
Submitted batch job 1539629
Submitted batch job 1539630


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1539631


...

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1539645
Submitted batch job 1539646


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1539647
Submitted batch job 1539648
Submitted batch job 1539649


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
